# Job creation and submission to Azure

**!Important!** 
This script uses **Azure SDKv2** for the creation of the training job. 
The notebook used for uploading the data uses **Azure SDKv1**

In [44]:
import os
from azure.ai.ml import MLClient, Input
from azure.identity import DefaultAzureCredential
from azure.ai.ml.entities import Environment
from azure.ai.ml import command

from src.utils import load_json, load_yaml

In [45]:
azure_connect_conf = load_json("./../../configs/azure_connect_example.json")
azure_connect_conf

{'subscription_id': 'bc82dccd-f19d-42cb-9ce3-0d5df33ef086',
 'resource_group': 'a0047-STUFIIT-ML01',
 'workspace_name': 'a0047stufiitml01'}

In [46]:
azure_job_conf = load_yaml("./../../configs/azure_job.yaml")
dataset_conf = azure_job_conf["dataset"]
environment_conf = azure_job_conf["environment"]
job_conf = azure_job_conf["job"]
job_args_conf = job_conf["args"]

In [47]:
# Authenticate and create a MLClient instance
credential = DefaultAzureCredential()
ml_client = MLClient(
    credential=credential,
    subscription_id=azure_connect_conf["subscription_id"],
    resource_group_name=azure_connect_conf["resource_group"],
    workspace_name=azure_connect_conf["workspace_name"]
)

Overriding of current TracerProvider is not allowed
Overriding of current LoggerProvider is not allowed
Overriding of current MeterProvider is not allowed
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented


In [48]:
# Check connection to workspace
ws = ml_client.workspaces.get(ml_client.workspace_name)
print(ws.name, ws.location, ws.resource_group, sep='\t')

a0047stufiitml01	westeurope	a0047-STUFIIT-ML01


In [49]:
# Get created Dataset as Data Asset
dataset = ml_client.data.get(name=dataset_conf["name"], version=dataset_conf["version"])

print(dataset.name)
print(dataset.path)

xsivakm1_tiger_segmentation
azureml://subscriptions/bc82dccd-f19d-42cb-9ce3-0d5df33ef086/resourcegroups/a0047-STUFIIT-ML01/workspaces/a0047stufiitml01/datastores/workspaceblobstore/paths/datasets/sivak-tiger/thrash/


In [50]:
# Prepare Environment
env = Environment(
    name=environment_conf["name"],
    description=environment_conf["description"],
    image=environment_conf["image"],
    conda_file=environment_conf["conda_file"]
)

print(env)

conda_file:
  channels:
  - defaults
  - conda-forge
  dependencies:
  - python=3.12
  - pip
  - pip:
    - torch
    - torchvision
    - pytorch-lightning
    - segmentation-models-pytorch
    - wandb
    - opencv-python-headless
    - pyyaml
    - json5
    - scikit-learn
  name: tiger-segmentation
description: For segmentation training on TIGER dataset
image: mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04
name: tiger-segmentation
tags: {}


In [51]:
# Define jobs
job = command(
    code=job_conf["code"],  # Project folder (contains main.py)
    command=f"python main.py "
            f"--data_path ${{{{inputs.input_data}}}} "
            f"--wandb {job_args_conf['wandb']} "
            f"--wandb_proj_name {job_args_conf['wandb_proj_name']} "
            f"--wandb_run_name {job_args_conf['wandb_run_name']} "
            f"--train_images_path {job_args_conf['train_images_path']} "
            f"--train_masks_path {job_args_conf['train_masks_path']} "
            f"--test_images_path {job_args_conf['test_images_path']} "
            f"--test_masks_path {job_args_conf['test_masks_path']} "
            f"--config {job_args_conf['config']}",
    inputs={
        "input_data": Input(
            type="uri_folder",
            path=dataset.path,
            mode="ro_mount"
            )
    },
    environment=env,
    experiment_name=job_conf["experiment_name"],
    compute=job_conf["compute"],  # compute cluster name
    display_name=job_conf["azure_display_name"]  # run display name
    )
        
# Submit the job
ml_client.jobs.create_or_update(job)

Uploading bc_project (0.02 MBs): 100%|██████████| 18953/18953 [00:00<00:00, 42561.44it/s]




Experiment,Name,Type,Status,Details Page
xsivakm1_segmentation,great_toe_q6mfgx0ztj,command,Starting,Link to Azure Machine Learning studio
